### Variational Autoencoder 

In [1]:
import warnings,os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim # I lkie slim 
import matplotlib.pyplot as plt
# warnings.filterwarnings("ignore") # Stop showing annoying warnings
tf.logging.set_verbosity(tf.logging.ERROR) # I like old-style MNIST w/o warnings
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.utils import shuffle
mnist = input_data.read_data_sets('data', one_hot=True)
%matplotlib inline
print ("TF version is %s."%(tf.__version__))

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
TF version is 1.8.0.


### Define vae class`

In [2]:
class vae(object):
    def __init__(self,_name='VAE',_xdim=784,_zdim=10,_hdim=128,_cdim=0,
                _actv=tf.nn.relu,_lr=0.001,_beta1=0.9,_beta2=0.9,_epsilon=0.1,
                _VERBOSE=True):
        self.name  = _name
        self.xdim  = _xdim
        self.zdim  = _zdim
        self.hdim  = _hdim
        self.cdim  = _cdim
        self.actv  = _actv
        self.lr    = _lr
        self.beta1 = _beta1
        self.beta2 = _beta2
        self.epsilon = _epsilon
        if _VERBOSE:
            print ("[%s] xdim:[%d] zdim:[%d] hdim:[%d] cdim:[%d]"\
                % (self.name,self.xdim,self.zdim,self.hdim,self.cdim))
        """ MODEL CONSTRUCTION """
        self._define_weights()
        self._build_model()
